In [8]:
!pip install datasets
!pip install tftrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification,AutoTokenizer,TFTrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from tftrainer import TrainArgument, Trainer

In [10]:
encodings = ['utf-8', 'latin-1', 'cp1252']
for encoding in encodings:
  try:
    df = pd.read_csv("/content/spam.csv", encoding=encoding)
    break
  except UnicodeDecodeError:
    pass

if 'df' not in locals():
  print("Couldn't decode the file using any of the specified encodings.")
else:

  print("Successfully read the CSV using", encoding)

Successfully read the CSV using latin-1


In [11]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [13]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
df= df.rename(columns={'v1':'label','v2':'message'})

In [15]:
y= list(df['label'])
X= list(df['message'])

In [16]:
encoder= LabelEncoder()
y= list(encoder.fit_transform(y))

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# **CLASSIFICATION USING TRANSFORMERS**

In [12]:
tokenizer= AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
train_encodings= tokenizer(X_train,truncation=True,padding=True)
test_encodings= tokenizer(X_test,truncation=True,padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
train_dataset= tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset= tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [15]:
model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Compile the model with loss and metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Train the model
history = model.fit(train_dataset.shuffle(1000).batch(16),
                    epochs=3,
                    batch_size=16,
                    validation_data=test_dataset.batch(16))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
279/279 [==============================] - 6978s 25s/step - loss: 0.0774 - sparse_categorical_accuracy: 0.9791 - val_loss: 0.0300 - val_sparse_categorical_accuracy: 0.9937
Epoch 2/3
279/279 [==============================] - 6805s 24s/step - loss: 0.0214 - sparse_categorical_accuracy: 0.9944 - val_loss: 0.0395 - val_sparse_categorical_accuracy: 0.9919
Epoch 3/3
279/279 [==============================] - 6718s 24s/step - loss: 0.0142 - sparse_categorical_accuracy: 0.9953 - val_loss: 0.0224 - val_sparse_categorical_accuracy: 0.9937


In [16]:
predictions = model.predict(test_dataset.batch(16))
predicted_labels = np.argmax(predictions.logits, axis=1)

70/70 [==============================] - 451s 6s/step


In [17]:
predicted_labels

array([0, 0, 1, ..., 0, 0, 1])

In [18]:
true_labels = np.array(y_test)

In [19]:
from sklearn.metrics import confusion_matrix, classification_report

cm= confusion_matrix(true_labels,predicted_labels)

In [20]:
print(classification_report(true_labels,predicted_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       965
           1       0.98      0.97      0.98       150

    accuracy                           0.99      1115
   macro avg       0.99      0.99      0.99      1115
weighted avg       0.99      0.99      0.99      1115

